# <center>Colorado K-12 Financial Transparency – <br>An Exploratory Data Science Comparison Across Schools</center>

### <center>MSDS 696: Data Science Practicum II - Stacey Sandy</center>

### <u><b>Overview </b></u>
This project is for the fulfillment of my final course in the pursuit of my Master of Science in Data Science degree at Regis University. In this past academic year, my kindergartner son has brought home 5 different fundraisers and a quarterly scholastic book fair flyer. I understand the scholastic book fair awards books to classes that purchase the most books. Unfortunately, I am unclear where the multiple fundraiser funding is going, what the funding is for, and how fundraisers feed into the school. These implications on the school and district fundraiser efforts need further analysis. Especially since school ends in a little over two months, I am curious to learn how expenses are handled. I intend to compare expenses across schools and then across districts. I would like to analyze which schools are spending more than others and why their expenses are more or less than others. I want to learn the cause and effects as to what are the trends on expenditures. Hopefully, I could learn more about why fundraising is important or necessary multiple times a year for any school. 

### <u><b>Objective</b></u>
The objective of this project and the primary research question is to address how financial expenditures are being spent across K through 12 schools in Colorado. An exploratory data science approach will be conducted to answer this research question. Overall, what are the school expenses that impact our children’s educational needs? Could the type of staffing, teacher types, and number of students be an impact? What are the contributing factors of the various expenses? How do schools and districts compare to one another? What variables are impacting the data reported? Could federal and state funding not be enough? 

### <u><b>Data Source</b></u>
* Colorado Department of Education website on Colorado’s K12 Financial Transparency website: https://coloradok12financialtransparency.com/#/
* The complete dataset of all Colorado K12 districts and BOCES can also be downloaded at: https://s3.amazonaws.com/bb-pub-pipeline-production/home/circleci/repo/public/uploads/data_files/uploads/000/000/416/20200114030354-colorado-financial-public-data-pipeline-v-1-Run_1-export-source-lep-facts.csv20200114-2678-rqjqmn.csv?1578999834

### <u><b>Financial Policies and Procedures Documentation</b></u>
* Colorado Department of Education. (2016, July 1). <i>Financial Policies and Procedures Chart of Accounts.</i> Retrieved from https://www.cde.state.co.us/cdefinance/fpp_coa1617

### <u><b>Things to take into consideration about the data:</b></u>
1. Data collected is for the Fiscal Year 2017-2018
    * Fiscal Year - a twelve-month accounting period (July 1 through June 30) to which the annual budget applies.
    * In this case, data is from July 1, 2017 through June 30, 2018
2. However, the financial data posted follows the standard <i>Financial Policies and Procedures Chart of Accounts</i> for the previous fiscal year 2016-2017.
<br>
This means the Financial Policies and Procedures Chart of Accounts for Fiscal Year 2017-2018, 2018-2019, and 2019-2020 may not coincide with the given data set provided from the Colorado K12 Financial Transparency website. This can be due to the fact that we are still in the current 2019-2020 Fiscal Year. Although, it is not mentioned or mad clear as to why the 2017-2018 Financial Policies and Procedures Chart of Accounts is not used from the archive. 
<br>

#### This project will examine codes that will only refer to the those codes applicable and found within the <u>2016-2017 Financial Policies and Procedures Chart of Accounts</u> as advised from the <u>Colorado K12 Financial Transparency</u> website FAQ. Should other information deem necessary, further research and examiniation may be conducted.
<br>
Resource<br>
Colorado Department of Education. (2019, June 13). <i>Chart of Accounts: Archive.</i> Copyright © 1999-2020 Colorado Department of Education. Retrieved from http://www.cde.state.co.us/cdefinance/sfcoa_archive
<br>
<br>
The excel document to match the data to the financial codes will be using of the "Chart_of_Account_1617.xlsx" document.

#### The following website is available for direct school or district searchs within Colorado:
* Colorado K12 Financial Transparency website: https://coloradok12financialtransparency.com/#/

### Project Process:
1. Conduct exploratory data analysis on complete dataset of all Colorado K12 districts and BOCES.<br>
2. Investigate unique Ids, values, and row/column data.<br>
3. Highlight financial codes of interest that are most used and least used.<br>
4. Add additional data from Colorado Department of Education on Financial Policies and Procedures Chart of Accounts.<br>
5. <br>
<br>
In final, compare and contrast findings. Answer research questions. Did you accomplish what your project objective detailed?

## <center>Exploratory Data Analysis (EDA) on all Colorado K12 districts and BOCES</center>

In [1]:
# Load libraries
import pandas as pd
from pandas_summary import DataFrameSummary
import numpy as np
from pandas_profiling import ProfileReport
import requests
import json

In [2]:
# Load the CO K12 dataset into a dataframe.
COK12_df = pd.read_csv('Data/colorado-financial-public-data-pipeline-v-1-Run#1-export-source-lep-facts.csv', header=4)

In [3]:
# Ensure columns were retrieved
COK12_df.columns

Index(['district_code', 'admin_unit', 'school_code', 'fund_code',
       'location_code', 'sre_code', 'program_code', 'object_source_code',
       'job_class_code', 'grant_code', 'amount_cents', 'exclude_school',
       'exclude_district'],
      dtype='object')

In [4]:
# Look at dataset head
COK12_df.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
0,3080,64203,0,10,600,0,20,100,200,0,71250.0,f,f
1,3080,64203,0,10,600,0,10,100,200,4010,13841274.0,f,f
2,3080,64203,0,10,700,0,2700,100,100,0,12171639.0,f,f
3,3080,64203,0,21,700,0,3100,100,100,0,4079700.0,f,f
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925.0,f,f


In [5]:
# Look at dataset tail
COK12_df.tail()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
280681,1180,64233,570,10,300,0,1800,810,0,0,340700.0,f,f
280682,1180,64233,429,11,951,0,2500,810,0,0,-700.0,f,f
280683,1180,64233,429,11,951,0,2600,810,0,0,45477.0,f,f
280684,1180,64233,429,11,951,0,2800,810,0,0,24500.0,f,f
280685,1180,64233,429,11,951,0,18,810,0,0,2499.0,f,f


In [6]:
# View dataframe shape (# of Rows and # of Columns)
COK12_df.shape

(280686, 13)

In [7]:
# Look at dataset information details
COK12_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280686 entries, 0 to 280685
Data columns (total 13 columns):
district_code         280686 non-null int64
admin_unit            280686 non-null int64
school_code           280686 non-null int64
fund_code             280686 non-null int64
location_code         280686 non-null int64
sre_code              280686 non-null int64
program_code          280686 non-null int64
object_source_code    280686 non-null int64
job_class_code        280686 non-null int64
grant_code            280686 non-null int64
amount_cents          280686 non-null float64
exclude_school        280686 non-null object
exclude_district      280686 non-null object
dtypes: float64(1), int64(10), object(2)
memory usage: 27.8+ MB


In [8]:
#Let's confirm a count on null valuws in the entire data set.
print(np.count_nonzero(COK12_df.isnull()))

0


In [9]:
# Look at the data frame summary data
COK12_dfs = DataFrameSummary(COK12_df)

In [10]:
COK12_dfs.columns_stats

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
counts,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686
uniques,200,62,1920,34,258,45,317,415,94,228,222337,2,2
missing,0,0,0,0,0,0,0,0,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
types,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,bool,bool


In [11]:
# Let's make a copy of the dataframe.
COK12_df2 = COK12_df.copy()

### Remember we must take note of the following information provided within the dataset (csv file)
<br>
When summarizing this information there are three important points to keep in mind:<br>
1) Fund 90 is used for informational purposes only and must be excluded from any analysis.<br>
2) All amounts are displayed with cents and without decimal places. E.g. the value $5.42 will be recorded as 542. <br>
3) The data methodology page contains details on the categories excluded from this summary at both the District and the School level.

In [12]:
# Drop the exclude_school and exclude_district columns from data frame
COK12_df2 = COK12_df2.drop(['exclude_school', 'exclude_district'],axis=1)

In [13]:
# Check columns again
COK12_df2.columns

Index(['district_code', 'admin_unit', 'school_code', 'fund_code',
       'location_code', 'sre_code', 'program_code', 'object_source_code',
       'job_class_code', 'grant_code', 'amount_cents'],
      dtype='object')

In [14]:
# Look at data frame head
COK12_df2.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250.0
1,3080,64203,0,10,600,0,10,100,200,4010,13841274.0
2,3080,64203,0,10,700,0,2700,100,100,0,12171639.0
3,3080,64203,0,21,700,0,3100,100,100,0,4079700.0
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925.0


### Note the trailing decimal and zero for the numbers in the amount_cents column. The amount_cents column is a float64 type as the df.info() details above.

In [15]:
# Change ammount_cents to int64 not float64
COK12_df2 = COK12_df2.astype({'amount_cents':'int64'})

In [16]:
# Confirm data types
COK12_df2.dtypes

district_code         int64
admin_unit            int64
school_code           int64
fund_code             int64
location_code         int64
sre_code              int64
program_code          int64
object_source_code    int64
job_class_code        int64
grant_code            int64
amount_cents          int64
dtype: object

In [17]:
# Look at tail and ensure amount_cents is no longer showing as a float type.
COK12_df2.tail()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
280681,1180,64233,570,10,300,0,1800,810,0,0,340700
280682,1180,64233,429,11,951,0,2500,810,0,0,-700
280683,1180,64233,429,11,951,0,2600,810,0,0,45477
280684,1180,64233,429,11,951,0,2800,810,0,0,24500
280685,1180,64233,429,11,951,0,18,810,0,0,2499


### Now, notice the numbers in the amount_cents column are showing as integers and whole numbers. We will have to think about adding decimals and commas to represent those column values as currency.

In [18]:
# Look at data frame shape
COK12_df2.shape

(280686, 11)

### Investigate the top 5 used district_codes
<br>
DEFINITION:
<br>
"District Code – a unique identifier tied to each reporting entity" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 5).
<br>
<br>
REMEMBER:
<BR>
"ONE METHOD for BOCES to track transactions by district being served is to define location codes such that each district has its own two-digit code.  Then these two-digit district codes may be combined with the appropriate first digit to create a location code that follows the structure from the chart of accounts" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 208).

In [19]:
# Let's see the top 5 used district_code values:
COK12_districtCodes = pd.value_counts(COK12_df2['district_code'].values,sort=True)
COK12_districtCodes.head()

880     34937
1420    21133
900     15183
20      11193
1550    10629
dtype: int64

<div class="alert alert-block alert-info">
    <b>Detail breakdown of the top 5 used District Codes:</b>
<br>
880: DENVER COUNTY 1
<br>
1420: JEFFERSON COUNTY R-1
<br>
900: DOUGLAS COUNTY RE 1
<br>
20: ADAMS 12 FIVE STAR SCHOOLS
<br>
1150: FREMONT RE-2
</div>

### Of the least used District Codes, code 9120 is ADAMS COUNTY BOCES as shown below.

In [20]:
# Look at tail of district codes
COK12_districtCodes.tail()

9140    71
9165    56
8042    42
8041    28
9120    22
dtype: int64

### Investigate the top 5 used admin_unit codes
<br>
DEFINITION:
<br>
"Administrative Unit – a unique identifier used to identify the specific administrative unit that the reporting unit is tied to" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 5).

REMEMBER:
<br>
"An Administrative Unit (AU) is a school district, board of cooperative services, or a combination of school districts or boards of cooperative services, which is approved by CDE and normally provides educational services to exceptional children.  AU’s function similarly to BOCES.  Thus, financial transactions between AU members and non-members are recorded using the same account codes and logic as financial transactions between BOCES members and non-members" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 205).

In [21]:
# Let's see the top 5 used admin_unit code values:
COK12_adminUnits = pd.value_counts(COK12_df2['admin_unit'].values,sort=True)
COK12_adminUnits.head()

16010    35049
30011    21133
18010    15183
1020     11193
35010    10629
dtype: int64

<div class="alert alert-block alert-info">
<b>Detail breakdown of the top 5 used Administrative Unit Codes:</b>
<br>
16010: Denver 1, Denver
<br>
30011: Jefferson R-1, Lakewood
<br>
18010: Douglas Re 1, Castle Rock
<br>
1020: Adams 12, Northglenn-Thornton
<br>
35010: Larimer R-1, Poudre
</div>

### The 5 least used Administrative Unit Codes shown below can <u>NOT</u> be found within the Chart of Account fiscal year 2016-2017.

In [22]:
# Look at admin units tail
COK12_adminUnits.tail()

19205    784
41010    678
62050    660
35030    625
49010    600
dtype: int64

### Investigate the top 5 used school_codes
<br>
DEFINITION:
<br>
"School Code – a unique identifier assigned for every school building at a school district for school site reporting" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 5).


### Investigate why the value <u>0</u> is the most commonly used school_code
NOTES on the use of the value <u>0</u>:
<br>
"Dimensions which are not required for a particular account type and are unused by the district, but are part of the basic account code structure, should be “zero filled" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 7).

In [23]:
# Let's see the top 5 used school_code values:
COK12_schoolCodes = pd.value_counts(COK12_df2['school_code'].values,sort=True)
COK12_schoolCodes.head(6)

0       68345
24       1163
4516      771
20        695
1426      496
7500      489
dtype: int64

<div class="alert alert-block alert-info">
    <b>Detail breakdown of the top 5 used School Codes:</b>
<br>
24: ADAMS CITY HIGH SCHOOL
<br>
4516: KEARNEY MIDDLE SCHOOL
<br>
20: ADAMS CITY MIDDLE SCHOOL
<br>
1426: CENTRAL ELEMENTARY SCHOOL
<br>
7500: ROSE HILL ELEMENTARY SCHOOL
</div>

### The 5 least used School Codes shown below can <u>NOT</u> be found within the Chart of Account fiscal year 2016-2017.

In [24]:
COK12_schoolCodes.tail()

2966    1
1455    1
4518    1
4520    1
6009    1
dtype: int64

### Investigate the top 5 used fund_codes
<br>
DEFINITION:
<br>
"Fund - an independent fiscal and accounting entity with a self-balancing set of accounts for recording cash and other financial resources.  It contains all related assets, liabilities and residual equities or balances, or changes therein.  Funds are established to carry on specific activities or attain certain objectives of the school district according to special legislation, regulations, or other restrictions" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 6).

In [25]:
# Let's see the top 5 used fund_code values:
COK12_fundCodes = pd.value_counts(COK12_df2['fund_code'].values,sort=True)
COK12_fundCodes.head()

10    156838
22     31148
11     24780
21     19035
23     16804
dtype: int64

<div class="alert alert-block alert-info">
    <b>Detail breakdown of the top 5 used Fund Codes:</b>
<br>
10: General Fund
<br>
22: Governmental Designated-Purpose Grants Fund
<br>
11: Charter School Fund
<br>
21: Food Service Fund
<br>
23: Pupil Activity Special Revenue Fund
</div>

### The 5 least used Fund Codes can <u>NOT</u> be found within the Chart of Account fiscal year 2016-2017.

In [26]:
# Look at fund codes tail
COK12_fundCodes.tail()

51    5
26    2
58    2
59    2
75    2
dtype: int64

### Investigate the top 5 used location_codes
<br>
DEFINITION:
<br>
"Location  - a dimension used as a way to identify schools, attendance centers, operational units, buildings, or sites as budgetary units or cost center designators, as a means of segregating costs" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 6).

NOTE:
<br>
"When compiling reports to reflect school district expenditures, the Colorado Department of Education (CDE) will use location codes in conjunction with program codes to report school district instructional programs by level.  However, the CDE will use program codes only--rather than program codes and location codes--to report school district support expenditures" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 19).

In [27]:
# Let's see the top 5 used location_code values:
COK12_locationCodes = pd.value_counts(COK12_df2['location_code'].values,sort=True)
COK12_locationCodes.head(6)

100    79496
300    39963
600    35268
200    29514
0      23168
500    15242
dtype: int64

### Investigate why the value <u>0</u> is amongst the top 5 most commonly used location_code

##### More NOTES on the use of the value <u>0</u> and notes specific to location_code:

"Note: Location codes 001 through 099 are available if needed for district use to identify elementary schools" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 19).

"A unique and specific location code should be used for each on-line school or program within the District" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 20).

"Note:  all on-line school and program location codes are bolded and on-line school and program detailed activities must be reported (reporting of single purchase service account for online is not allowable" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 20).

"900-969  Location codes 900 through 969 are reserved for district use for charter schools only.<br>
900 – 919  Elementary Charter Schools.  Location codes 900 through 919 must be used for elementary charter schools.<br>
920 – 929 Middle School Charter Schools.  Must be used for middle school (junior high) charter schools.<br>
930 – 939 High School Charter Schools.  Must be used for high school charter schools.<br>
940 – 949 Vocational/Technical Charter Schools.  Must be used for vocational/technical charter schools.<br>
950 – 969  Combination Charter Schools.  Location codes 950 through 969 must be used for combination charter schools
970 Other Operational Units.<br>
Location codes 970 through 989 are available for district use for other operational units not specified elsewhere.<br>
990 Private/Parochial Schools.  Used to identify private or parochial schools.<br>
Location codes 990-999 are available for district use." (CDE, <i>FY16-17 Chart of Accounts</i>, p. 21).
<br>
<br>
<b>"In order to produce an apples-to-apples comparison, CDE will use location codes to aggregate instructional program data reported for elementary and middle schools regardless of whether these data are reported by subject or by a general education program such as 0070 Gifted and Talented or 0080 General Instructional Media" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 28).</b>
<br>
<br>
"Charter school location codes MUST be used with ALL transactions relating to charter schools" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 149).
<br>
<br>
<b>"HOW LOCATION CODES CAN BE USED BY THE BOCES 
<br>
CDE will use location codes numbered 100 through 599 in conjunction with instructional program codes to identify instructional costs by level (i.e., as elementary, middle/junior high school, or high school). Therefore, BOCES MUST use the structure for location codes that is defined in the chart of accounts" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 207).</b>
<br>
<br>
"ONE METHOD for BOCES to track transactions by district being served is to define location codes such that each district has its own two-digit code.  Then these two-digit district codes may be combined with the appropriate first digit to create a location code that follows the structure from the chart of accounts" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 208).
<br>
<br>
"LOCATIONS

Locations 900 through 969.

Location codes 900 through 969, which are reserved exclusively for charter schools, are required for recording charter school revenues and expenditures and balance sheet.  Each charter school must have a unique location code that identifies the school and level.  So that the level can be identified, the following structure must be used when assigning location codes.   

* Locations 900 through 919 are used for elementary charter schools.
* Locations 920 through 929 are used for middle school (jar high) charter schools. 
* Locations 930 through 939 are used for high school charter schools.
* Locations 940 through 949 are used for vocational/technical charter schools.
* Locations 950 through 969 are used for combination charter schools. 

These charter school location codes must be used for all charter school transactions regardless of fund" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 235).

"Note: Even though location codes and program codes are not required for revenue transactions, the district may find use of location codes and/or program codes in user fee revenue account strings helpful for identifying usage by specific programs or sites" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 248).

"Note: Even though location codes and program codes are not required for revenue transactions, the district may find use of location codes and/or program codes in user fee revenue account strings helpful for identifying usage by specific programs or sites" (CDE, <i>FY16-17 Chart of Accounts</i>, p.255).

<div class="alert alert-block alert-info">
<b>Detail breakdown of the top 5 used Location Codes:</b>
<br>
100: Location Codes <b>100 - 199</b> is used for Elementary Schools per the source (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: Location).
<br>
300: Location Codes <b>300 - 399</b> is used for High Schools per the source (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: Location).
<br>
600: Centralized Services <b>600 - 699</b> is used for "Activities or services associated with district-wide or centralized administration, program direction, curriculum services, business services, and related support services" (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: Location).
<br>
200: Location Codes <b>200 - 299</b> is used for Middle (Junior High) Schools per source (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: Location).
<br>
<b>0: Location_code <u>Zero "0"</u> account for 23,168 financially funded items.</b>
<br>
500: Combination Schools <b>500 - 599</b> is used for "Schools that house on a permanent basis more than one type of instructional organization; e.g. an elementary school and a middle school" (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: Location).
</div>

### Let's look at the 5 least used Location Codes: 

In [28]:
# Look at tail of location codes
COK12_locationCodes.tail()

182    2
104    2
134    2
623    2
983    1
dtype: int64

### Of the 5 least used Location Codes shown below, the fewest Program Codes used are as follows:
<br>
182: Elementary Schools
<br>
104: Elementary Schools
<br>
134: Elementary Schools
<br>
623: Centralized Services
<br>
983: Other Operational Units
<br>
As quoted, "Location codes 970 through 989 are available for district use for other operational units not specified elsewhere" (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: Location).

### Investigate the top 5 used sre_codes

DEFINITION:
<br>
"Special Reporting Element (SRE) - describes the activity for which a service or material object is acquired, much like a function.  This element is optional, but may be used with the program element to differentiate program costs" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 6).

REMEMBER:
<BR>
"Note: SRE codes 60 through 89 are available for district use" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 25).

### Investigate why <u>0</u> is the most used sre_code

"...the Special Reporting Element describes the activity for which a service or material object is acquired.  The SRE’s of a school district are classified into six broad areas:  Instruction, Support Services, Non-Instructional Services, Facilities Acquisition and Construction Services, Other Uses, and Reserves.  SREs and sub-SREs consist of activities which have somewhat the same general operational objectives.  Categories of activities comprising each of the divisions and subdivisions are grouped according to the principle that the activities can be combined, compared, related, and mutually exclusive.

<B>The use of the SRE is voluntary.  Nonetheless and as with all non-required (non-bolded) items, it is recommended that if a district chooses to implement this element,tot follow the scheme provided...</B>

Note: the one non-optional exception to this is SRE code 95.  SRE 95 is used to identify Consolidated School wide Plans" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 23).

In [29]:
# Let's see the top 5 used sre_code values:
COK12_sreCodes = pd.value_counts(COK12_df2['sre_code'].values,sort=True)
COK12_sreCodes.head(6)

0     268470
60      3035
95      2775
21       704
22       580
11       546
dtype: int64

<div class="alert alert-block alert-info">
    <b>Detail breakdown of the top 5 used Special Reporting Element (SRE) Codes:</b>
<b>0: sre_code <u>Zero "0"</u> account for 268,470 financially funded Special Reporting Elements (SRE) items.</b>
<br>
60: <b>Other</b> SRE Code is said to be used for "SRE codes 60 through 89 are available for district use"  (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE).
<br>
95: <b>Consolidated School wide Program</b> is said to be used for "This SRE code identifies Consolidated School wide Programs.  It is used in conjunction with a unique location code for each school wide plan site and Object Code 0855.  Object Code 0855 identifies the School wide Plan Distribution, as an option to detailed object coding.  SRE 95 should be used for the coding of all expenditure activities related to a specific school’s school wide plan.  SRE should also be used in all General Fund (10) Abatement Accounts and all Governmental Grants Fund (22) or General Fund (10) Grant Accounts for the grant programs that have been included in the specific school wide plan. See Appendix T, “Consolidated School wide Programs,” for additional information" (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE). 
<br>
21: <b>Students</b> is said to be used for "Activities designed to assess and improve the well-being of students and to supplement the teaching process" (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE). 
<br>
22: <b>Instructional Staff</b> is said to be sued for " Activities associated with assisting the instructional staff with the content and process of providing learning experiences for students." (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE). 
<br>
11: <b>Regular Education</b> *There are no other detail information provided on what Regular Education entails.
</div>

### Let's look at the least 5 used SRE Codes: 

In [30]:
# Look at SRE Code tail
COK12_sreCodes.tail()

75    3
42    2
7     1
1     1
29    1
dtype: int64

### Of the 5 least used SRE Codes used, the fewest sre codes used are as follows:
<br>
75: <b>Other</b> SRE Code is said to be used for "SRE codes 60 through 89 are available for district use"  (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE).
<br>
42: <b>Site Improvement Services</b> is said to be used for "Activities concerned with improving sites, and with maintaining existing site improvements" (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE).
<br>
7: <b>Not found within the Chart of Accounts fiscal year 2016-2017.</b>
<br>
1: <b>Not found within the Chart of Accounts fiscal year 2016-2017.</b>
<br>
29: <b>Other</b> is said toe be used for, "All other support services not classified elsewhere in the 20 series" "  (CDE, <i>Chart of Accounts_1617.xlsx</i>, sheet: SRE).

### Investigate the top 5 used program_code

DEFINITION:
<br>
"Program - a dimension which describes the activity for which a service or material object is acquired. The programs of a school district are classified into six broad areas:  Instruction, Support Services, Operation of Non-Instructional Services, Facilities Acquisition and Construction Services, Other Uses, and Reserves" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 6).

REMEMBER:
<BR>
"A program is a plan of activities and procedures designed to accomplish a predetermined objective or set of objectives. The program dimension provides the school district a framework to classify expenditures to determine total costs of attaining such predetermined objectives.  The program dimension allows a school district to charge costs, instructional and support, directly to the benefiting program" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 27).

### Investigate why <u>0</u> is the most used program_code

In [31]:
# Let's see the top 5 used program_code values:
COK12_programCodes = pd.value_counts(COK12_df2['program_code'].values,sort=True)
COK12_programCodes.head(6)

0       39439
2400    21585
10      19640
2200    18478
2100    17312
1700    14123
dtype: int64

<div class="alert alert-block alert-info">
<b>Detail breakdown of the top 5 used Program Codes:</b>
<b>Zero "0" program_code account for 39,439 financially funded items.</b>
<br>
2400: Support Services - School Administration
<br>
10: General Elementary Education
<br>
2200: Support Services - Instructional Staff
<br>
2100: Support Services - Students
<br>
1700: Special Education
</div>

### Look at the least 5 used Program Codes used:

In [33]:
# Look at program code tail
COK12_programCodes.tail()

1969    1
1771    1
1972    1
2122    1
2161    1
dtype: int64

### Of the 5 least used Program Codes shown below, the fewest Program Codes used are as follows:
<br>
1969: Not found within the Chart of Accounts fiscal year 2016-2017. 
<br>
1771: Speech Pathology Services
<br>
1972: Not found within the Chart of Accounts fiscal year 2016-2017.
<br>
2122: Counseling Services
<br>
29: Not found within the Chart of Accounts fiscal year 2016-2017.

### Investigate the top 5 used object_source_code

DEFINITION:
<br>
"Source/Object/Balance Sheet - a combination dimension which is used to identify the type of account: Balance Sheet, Revenue (Sources), or Expenditure (Object).  Object is the service or commodity obtained" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 6).

REMEMBER:
<br>
Within the Colorado Department of Education Financial Prolicies and Procedures Chart of Accounts (fiscal year 2016-2017), Object Source Code or Source/Object code are defined as an account code element:
<br>

In [34]:
# # Let's see the top 5 used sre_code values:
COK12_objectSourceCodes = pd.value_counts(COK12_df2['object_source_code'].values,sort=True)
COK12_objectSourceCodes.head()

100    50875
200    50215
600    33490
500    11008
580    10824
dtype: int64

<div class="alert alert-block alert-info">
<b>Detail breakdown of the top 5 used Object Source Balance Codes:</b>
<br>
100: Salaries
<br>
200: Employee Benefits
<br>
600: Supplies
<br>
500: Other Purchased Services
<br>
580: Travel, Registration, and Entrance
</div>

### Revenue, Expenditure, and Balance Sheet Account Code Structure
<br>
NOTE:

"The following illustrations present the basic account code structure for each account type by a series of Xs and hyphens.  Each X designates the placement of a numerical digit (which can include zero) in the account code" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 8). (More information later on the account code structure.)

"Offset to Source Codes 1956, 3956, and 4956, used with the appropriate grant code, which may be zero" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 134).

"Offset to Object Code 0596, used with the appropriate grant code, which may be zero" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 151).

"Offset to Source Codes 1956, 3956, and 4956, used with the appropriate grant code, which may be zero" (CDE, <i>FY16-17 Chart of Accounts</i>, p. 257).
<br>
<br>
All three account types—revenues, expenditures, and balance sheet accounts—use the same basic multidimensional account code structure as shown below:

## <center><u>Format of Accounts</u></center>

### This is the format and sequencing of the elements in the REVENUE account code:
![Revenue](Images/revCOA.PNG)

### This is the format and sequencing of the elements in the EXPENDITURE account code:
![Expenditure](Images/expCOA.PNG)

### This is the format and sequencing of the elements in the BALANCE account code:
![Balance](Images/bsCOA.PNG)

<br>
(CDE, <i>FY16-17 Chart of Accounts</i> pg. 7, 8, and 9).
<br>
<br>
Reference:
<br>
<br>
Colorado Department of Education. (2016, July 1). Chart of Accounts: 2016-17. Copyright © 1999-2020 Colorado Department of Education. Retrieved from https://www.cde.state.co.us/cdefinance/fpp_coa1617 

#### Let's look at the least 5 Object Source Balance Codes used: 

In [35]:
# Look at object source code tail
COK12_objectSourceCodes.tail()

3240    1
1810    1
7441    1
618     1
0       1
dtype: int64

#### Of the 5 least used Object Source Balance Codes shown below, the fewest Object Source Balance Codes used are as follows:
<br>
3240: Not found within the Chart of Accounts fiscal year 2016-2017. 
<br>
1810: Adult Education
<br>
7441: Matured bonds payable
<br>
618: Not found within the Chart of Accounts fiscal year 2016-2017.
<br>
0: <b>object_source_code <u>Zero "0"</u> account for 1 financially funded Object/Source/Balance code.</b>

## STOPPED HERE!

In [36]:
# Let's count how many differentjob_class_code values are in each column. 
COK12_jobClassCodes = pd.value_counts(COK12_df2['job_class_code'].values,sort=True)
COK12_jobClassCodes.head(6)

0      172392
200     52661
400     21075
600      7722
500      7504
100      7274
dtype: int64

### Investigate why <u>0</u> is the largest used job_class_code!?!

<div class="alert alert-block alert-info">
<b>Detail breakdown of the top 5 used Job Class Codes:</b>
<br>
0: <b>Zero "0" job_class_code account for 172,392 financially funded items.</b>
<br>
200: Professional - Instructional
<br>
400: Paraprofessionals
<br>
600: Not found within the Chart of Accounts fiscal year 2016-2017. 
<br>
500: Not found within the Chart of Accounts fiscal year 2016-2017. 
<br>
100: Administrator
</div>

#### Okay, let's look at the least 5 Job Class Codes used: 

In [37]:
# Look at job class code tail
COK12_jobClassCodes.tail()

630    3
634    3
601    2
401    2
417    2
dtype: int64

#### Of the Job Class Codes used, the fewest job_class codes used are as follows:
<br>
630: Vehicle Operator
<br>
634: Student Worker
<br>
601: Brick Mason 
<br>
401: Bilingual Assistant
<br>
417: Teaching/Classroom Technician

In [38]:
# Let's count how many different grant_code values are in each column. 
COK12_grantCodes = pd.value_counts(COK12_df2['grant_code'].values,sort=True)
COK12_grantCodes.head(6)

0       197260
3130     11884
4010      9629
1         8084
3120      4242
3140      4095
dtype: int64

### Investigate why <u>0</u> is the largest used grant_code!?!

<div class="alert alert-block alert-info">
<b>Detail breakdown of the top 5 used Grant Codes</b>
<br>    
0: <b>Zero "0" grant_code account for 197,260 financially funded items.</b>
<br>
3130: State ECEA (Special Education)
<br>
4010: No Child Left Behind, Title I, Part A: Improving Basic Programs Operated by Schools. Formula Grant CFDA# 84.010
<br>
1: Local and Intermediate Project/Grants
<br>
3120: State Vocational Education (Source 3010)
<br>
3140: State ELPA
</div>

#### Now look at the least 5 Grant Codes used: 

In [39]:
# Look at grant code tail
COK12_grantCodes.tail()

3162    1
7389    1
9005    1
1787    1
9036    1
dtype: int64

#### Of the Grant Codes used, the fewest grant codes used are as follows:
<br>
3162: School Breakfast Program 
<br>
7389: ARRA - Title I, Part A Delinquent CFDA# 84.389
<br>
9005: Department of Defence Supplemental CFDA# None
<br>
1787: Not found within the Chart of Accounts fiscal year 2016-2017.
<br>
9036: Disaster Grants Public Assistance (Presidential Declared Disasters) CFDA# 97.036

In [39]:
# Let's groupBy object_source_code codes.
COK12_groupByOSC = COK12_df2.groupby('object_source_code')
COK12_groupByOSC.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250
1,3080,64203,0,10,600,0,10,100,200,4010,13841274
2,3080,64203,0,10,700,0,2700,100,100,0,12171639
3,3080,64203,0,21,700,0,3100,100,100,0,4079700
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925
...,...,...,...,...,...,...,...,...,...,...,...
156473,3148,64203,6812,10,800,0,2850,525,0,0,206878
156518,3148,64203,6812,21,700,0,3100,631,0,0,264425
156521,3148,64203,6812,21,700,0,3100,634,0,0,629105
189855,880,16010,8945,11,966,0,2700,512,0,0,1278176


In [40]:
# Try to melt function to identify each object source code by other financial code columns
longOSC = pd.melt(COK12_df2, 
    id_vars=['object_source_code'])

In [41]:
longOSC.head()

,object_source_code,variable,value
0,100,district_code,3080
1,100,district_code,3080
2,100,district_code,3080
3,100,district_code,3080
4,100,district_code,3080


In [42]:
longOSC.tail()

,object_source_code,variable,value
2806855,810,amount_cents,340700
2806856,810,amount_cents,-700
2806857,810,amount_cents,45477
2806858,810,amount_cents,24500
2806859,810,amount_cents,2499


### <center>Begin Pre-liminary Visualizations</center>

In [43]:
# Import libraries
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
COK12_df2.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250
1,3080,64203,0,10,600,0,10,100,200,4010,13841274
2,3080,64203,0,10,700,0,2700,100,100,0,12171639
3,3080,64203,0,21,700,0,3100,100,100,0,4079700
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925


In [45]:
# Insert a visualization here for more depth
plt.barh(COK12_df2.fund_code, COK12_df2.district_code)
plt.xlabel('District Code')
plt.ylabel('Fund Code')
plt.title('District Codes for Each Fund Code')
plt.savefig('DC_FC.png', dpi=300, bbox_inches='tight')

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x00000248896B5438> (for post_execute):


KeyboardInterrupt: 

### Investigate the <u>specific</u> object source codes!

In [46]:
# Let's try to group the fund code, location code, and program code
groupFLP = COK12_df2.groupby(['fund_code', 'location_code', 'program_code'])
groupFLP.max()

district_code  admin_unit  school_code  \
fund_code location_code program_code                                           
6         0             0                      3000       59010            0   
          600           2200                   3000       59010            0   
                        2300                   3000       59010            0   
                        2800                   3000       59010            0   
          700           2600                   3000       59010            0   
...                                             ...         ...          ...   
85        971           0                       880       16010            0   
                        3300                    880       16010            0   
90        0             0                      3230       64233         9360   
          600           0                       123        3020            0   
          800           0                      3140       64193            0   

                                      sre_code  object_source_code  \
fund_code location_code program_code                                 
6         0             0                    0                8121   
          600           2200                 0                 730   
                        2300                 0                 300   
                        2800                 0                 500   
          700           2600                 0                 400   
...                                        ...                 ...   
85        971           0                    0                8242   
                        3300                 0                 800   
90        0             0                    0                7519   
          600           0                    0                7515   
          800           0                    0                7515   

                                      job_class_code  grant_code  amount_cents  
fund_code location_code program_code                                            
6         0             0                          0           0     290050022  
          600           2200                       0           0      18494795  
                        2300                       0           0        463561  
                        2800                       0           0      16749358  
          700           2600                       0           0       4800000  
...                                              ...         ...           ...  
85        971           0                          0           0    1271687000  
                        3300                       0           0     797166000  
90        0             0                          0           0   57318500000  
          600           0                          0           0    2193697300  
          800           0                          0           0  178359000000  

[6153 rows x 8 columns]

In [47]:
# Try to melt function to identify each fund code by other financial code columns
longFund = pd.melt(COK12_df2, 
    id_vars=['fund_code'])

In [48]:
longFund.head()

,fund_code,variable,value
0,10,district_code,3080
1,10,district_code,3080
2,10,district_code,3080
3,21,district_code,3080
4,10,district_code,3080


In [49]:
longFund.tail()

,fund_code,variable,value
2806855,10,amount_cents,340700
2806856,11,amount_cents,-700
2806857,11,amount_cents,45477
2806858,11,amount_cents,24500
2806859,11,amount_cents,2499


In [ ]:
# Insert a visualization here for more depth

### Investigate <u>fund codes</u>, <u>location codes</u>, and <u>program codes</u>!!

In [50]:
# Try to melt function to identify each fund code by other financial code columns
longLC = pd.melt(COK12_df2, 
    id_vars=['location_code'])

In [51]:
longLC.head()

,location_code,variable,value
0,600,district_code,3080
1,600,district_code,3080
2,700,district_code,3080
3,700,district_code,3080
4,200,district_code,3080


In [52]:
longLC.tail()

,location_code,variable,value
2806855,300,amount_cents,340700
2806856,951,amount_cents,-700
2806857,951,amount_cents,45477
2806858,951,amount_cents,24500
2806859,951,amount_cents,2499


In [ ]:
# Insert a visualization here for more depth

In [53]:
# Try to melt function to identify each fund code by other financial code columns
longPC = pd.melt(COK12_df2, 
    id_vars=['program_code'])

In [54]:
longPC.head()

,program_code,variable,value
0,20,district_code,3080
1,10,district_code,3080
2,2700,district_code,3080
3,3100,district_code,3080
4,1700,district_code,3080


In [55]:
longPC.tail()

,program_code,variable,value
2806855,1800,amount_cents,340700
2806856,2500,amount_cents,-700
2806857,2600,amount_cents,45477
2806858,2800,amount_cents,24500
2806859,18,amount_cents,2499


In [ ]:
# Insert a visualization here for more depth

# LOAD FFP Chart of Accounts for 2016-2017 to merge with the data:

In [56]:
# Load the Financial Policies and Procedures Chart of Accounts 2016-2017 into a dataframe but by District Code sheet
COA1617_DC = pd.read_excel('Data/Chart of Accounts_1617.xlsx', sheet_name='District Code', header=0)

In [57]:
COA1617_DC.head()


,District,Description,Detailed Description,
0,10,MAPLETON 1,MAPLETON 1,NaN
1,20,ADAMS 12 FIVE STAR SCHOOLS,ADAMS 12 FIVE STAR SCHOOLS,NaN
2,30,ADAMS COUNTY 14,ADAMS COUNTY 14,NaN
3,40,SCHOOL DISTRICT 27J,SCHOOL DISTRICT 27J,NaN
4,50,BENNETT 29J,BENNETT 29J,NaN


In [58]:
COA1617_DC.tail()

,District,Description,Detailed Description,
196,9165,UTE PASS BOCES,UTE PASS BOCES,NaN
197,9170,COLORADO DIGITAL BOCES,COLORADO DIGITAL BOCES,NaN
198,8001,CHARTER SCHOOL INSTITUTE,CHARTER SCHOOL INSTITUTE,NaN
199,9000,Colorado School for the Deaf and Blind,Colorado School for the Deaf and Blind,NaN
200,9999,FOR DISTRICT WIDE SCHOOL (9980),FOR DISTRICT WIDE SCHOOL (9980),NaN


In [59]:
#Load excel spreadsheet without destinguishing sheet name
COA1617_ALL = pd.read_excel('Data/Chart of Accounts_1617.xlsx', header=0)

In [60]:
# Check head
COA1617_ALL.head()

,Unnamed: 0,EXPENDITURE,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,XXXX,-,XXXXX,-,XXXX,-,XX,-,XXX,-,XX,-,XXXX,-,XXXX,-,XXX,-,XXXX
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
#Check tail...
COA1617_ALL.tail()

,Unnamed: 0,EXPENDITURE,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PROGRAM,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BALANCE SHEET,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPTIONAL,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DESIGNATED GRANT\n PROJECT


### Looks a bit messy without loading each sheet from the excel spread sheet individually..
<br>
Try to load each sheet instead.....

In [ ]:
# To Do:
# Research code to add decimal to amount_cents into dollar type or dollar value.
# Research more on how best to aggregate and group relevent data.
# Begin adding visualizations based on the relevant financial data to present.
# Begin first phases of KNN and DBSCAN

In [ ]:
# Testing out locale and babel for working with currency

In [62]:
import locale

In [63]:
from babel.numbers import format_currency

In [64]:
COK12_df2.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250
1,3080,64203,0,10,600,0,10,100,200,4010,13841274
2,3080,64203,0,10,700,0,2700,100,100,0,12171639
3,3080,64203,0,21,700,0,3100,100,100,0,4079700
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925


In [65]:
COK12_df2.dtypes

district_code         int64
admin_unit            int64
school_code           int64
fund_code             int64
location_code         int64
sre_code              int64
program_code          int64
object_source_code    int64
job_class_code        int64
grant_code            int64
amount_cents          int64
dtype: object

In [66]:
# Try PrettyPandas
# source: https://prettypandas.readthedocs.io/en/latest/quickstart.html
from prettypandas import PrettyPandas

In [67]:
# Attempt using pretty pandas currency attribute
COK12_df3 = PrettyPandas(COK12_df2).as_currency(subset='amount_cents', currency=u'USD', locale=('en_US')) # Format just amount_cents column

In [68]:
# Check head of data frame
COK12_df3.head()

AttributeError: 'PrettyPandas' object has no attribute 'head'

In [69]:
# Take a look at Pretty Pandas type, Is it actually a data frame?
type(COK12_df3)

prettypandas.summarizer.PrettyPandas

### head() will not work on pretty pandas "summarizer" type...

In [ ]:
# WARNING!!! This takes a long time to run...
#Check to see if the amount_cents column converted to a currency type
COK12_df3

In [84]:
# Convert the pretty pandas summarizer back to a pandas data frame
COK12_df4 = COK12_df3.to_frame()

In [85]:
type(COK12_df4)

pandas.core.frame.DataFrame

### This is when head() will work on a pandas data frame not a pretty pandas summarizer type....
<br>
Investigate how to change amount_cents column into a <b>currency type</b>....

In [86]:
COK12_df4.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250
1,3080,64203,0,10,600,0,10,100,200,4010,13841274
2,3080,64203,0,10,700,0,2700,100,100,0,12171639
3,3080,64203,0,21,700,0,3100,100,100,0,4079700
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925


### Why is the amount_cents column not converting to currency??

Thanks to Professor Nathan George, he provided a helpful function to convert the column amount_cents to string and then back to float with the decimal in the correct place for the last 2 digits as the cents part of the currency value.

In [87]:
# Function to convert integer to string, add decimal, and convert bck to float. (Courtesy of Professor George)

def convert_to_decimal(number):
    # convert number to string
    num_str = str(number)

    # take everything but the last two digits as the integer part of the number
    int_part = num_str[:-2]

    # take the last 2 digits as the decimal places
    dec_part = num_str[-2:]
    
    return int_part + dec_part

In [94]:
COK12_df4['amount_cents'] = COK12_df4.apply['amount_cents'].apply(convert_to_decimal)

TypeError: 'method' object is not subscriptable

In [97]:
COK12_df4.amount_cents(convert_to_decimal)

TypeError: 'Series' object is not callable

In [98]:
type(COK12_df4)

pandas.core.frame.DataFrame

In [95]:
COK12_df4.dtypes

district_code         int64
admin_unit            int64
school_code           int64
fund_code             int64
location_code         int64
sre_code              int64
program_code          int64
object_source_code    int64
job_class_code        int64
grant_code            int64
amount_cents          int64
dtype: object

In [96]:
COK12_df4.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250
1,3080,64203,0,10,600,0,10,100,200,4010,13841274
2,3080,64203,0,10,700,0,2700,100,100,0,12171639
3,3080,64203,0,21,700,0,3100,100,100,0,4079700
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925


In [58]:
# Continue working on how to change amount_cents column into currency ex: 712.50

In [ ]:
# Stopped code here

## <center>Attempt feature engineering</center>
<br>
Column 11 amount_cents is the target variable.
<br>
<br>
I will attempt to:<br>
* Conduct a 80 / 20 train-test split<br>
* Train a RandomForestClassifier<br>
* Look at the feature importance of each column to the target value<br>
* Run & evaluate accuracy on the test data<br>

In [99]:
#Import scikit learn library
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [100]:
# Assign y value to target value and drop from original date frame
y = COK12_df2.amount_cents
x = COK12_df2.drop(['amount_cents'], axis=1)

In [101]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [102]:
clf=RandomForestClassifier(n_estimators=100)

In [103]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for column_name in x_train.columns:
        if x_train[column_name].dtype == object:
            x_train[column_name] = le.fit_transform(x_train[column_name])
        else:
            pass

In [104]:
clf.fit(x_train,y_train)

MemoryError: could not allocate 10520100864 bytes

### Find out my there is a MemoryError??? as listed above???

## <center>START VISUALIZATIONS</center>

In [ ]:
# Import libraries
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Poor attempt at paandas profile report....

In [13]:
# Generate pandas profile report
COK12_profile = ProfileReport(COK12_df, title='Pandas CO K12 Profiling Report', html={'style':{'full_width':True}})

TypeError: _plot_histogram() got an unexpected keyword argument 'title'

In [ ]:
# Display profile report in Jupyter Notebook widgets
COK12_profile.to_widgets()

In [ ]:
# Include html report in Jupyter Noteboook
COK12_profile.to_notebook_iframe()

In [ ]:
# Save the profile report out to html
COK12_profile.to_file(output_file="your_report.html")